In [38]:
# Treasury Forecasting algo version #2 determining the best investnent windows
import mysql.connector
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
load_dotenv(verbose=True)

# Database connection parameters
DB_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "database": os.getenv("DB_NAME"),
}

print('Database URL: ' + os.environ.get('DB_HOST'))

Database URL: timetables.mysql.database.azure.com


In [39]:
# Fetch data from database and return as a DataFrame
def fetch_data(table_name, column_names='*', condition='1', sql=False):
    try:
        # Connect
        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()
        # Fetch
        if sql:
            cursor.execute(sql)
        else:
            query = f"SELECT {column_names} FROM {table_name} WHERE {condition}"
            cursor.execute(query)
        # Fetch column names
        columns = [col[0] for col in cursor.description]
        # Fetch data
        data = cursor.fetchall()
        df = pd.DataFrame(data, columns=columns)
        return df
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if 'conn' in locals() and conn.is_connected():
            cursor.close()
            conn.close()

In [40]:
# STEP 1: Get the asset classes combined with their parent class ID
table_name = 'AssetClass'
column_names = ''
condition = ''
sql = """SELECT a.`ID`, a.`Title`, a.`Group`, a.`Issuer`, a.`PercentMax`,
CASE WHEN p.`Title` IS NULL THEN a.`Title` ELSE p.`Title` END AS `AssetClassCombined`
FROM `AssetClass` a
LEFT JOIN ( SELECT ac.`ID`, ac.`Title`, ac.`Group`, ac.`PercentMax` FROM `AssetClass` ac
WHERE AssetClassParentID = 0 ) p ON p.ID = a.AssetClassParentID
WHERE AssetClassParentID = 0 AND a.Title != 'Not Assigned'
"""
assetClasses = fetch_data(table_name, column_names,1,sql)
assetClasses

,ID,Title,Group,Issuer,PercentMax,AssetClassCombined
0,1,Cash/Sweep,Not Assigned,None,1.0000,Cash/Sweep
1,2,Certificate of Deposit,Certificates of Deposit,None,0.5000,Certificate of Deposit
2,3,Commercial Paper,Commercial Paper,None,0.3330,Commercial Paper
3,4,US Agencies,US Agencies,None,0.2000,US Agencies
4,7,Money Market,Not Assigned,None,0.2000,Money Market
5,8,Mutual Fund,Not Assigned,None,0.2000,Mutual Fund
6,16,US Treasuries,US Treasuries,None,1.0000,US Treasuries


In [ ]:
# STEP 2: Get the transactions with parent classes from the transactions view
#         This view is created in the database to combine the transactions with their
#         parent classes and respects the date range specified in the application
table_name = 'TransactionView'
transactions_df = fetch_data(table_name)
transactions_df.fillna(0, inplace=True)
# Convert specified columns to int type
transactions_df[['TransactionTypeID', 'AssetID', 'AssetClassID']] = transactions_df[['TransactionTypeID', 'AssetID', 'AssetClassID']].astype(int)
transactions_df

,ID,TransactionDate,Notes,FinancialFlow,TransactionTypeID,OperationTypeID,Amount,AssetID,AssetClassID,TransactionClass
0,13643,2025-01-22,0,Outflow,18,3,-130000.00,0,0,Cash/Sweep
1,13644,2025-01-22,Oct-24,Outflow,49,3,-165373000.00,0,0,Cash/Sweep
2,13645,2025-01-22,State Bonds,Inflow,25,2,3395000.00,0,0,Cash/Sweep
3,13938,2025-01-22,25007 313385AX4,Inflow,0,4,11000000.00,8058,17,US Agencies
4,13948,2025-01-22,25020 313589AX1,Inflow,0,4,22980000.00,8063,6,US Agencies
...,...,...,...,...,...,...,...,...,...,...
256,100003,2025-01-21,Imported balance,Inflow,41,11,-170669204.47,0,0,Commercial Paper
257,100009,2025-01-21,Imported balance,Inflow,0,11,-151070374.54,0,0,Cash/Sweep
258,100009,2025-01-21,Imported balance,Inflow,0,11,-18975581.76,0,0,Mutual Fund
259,100003,2025-01-21,Imported balance,Inflow,0,11,-20065580.67,0,0,Money Market
